# Benchmarking cifar-10 and cifar-100

## This is a work in progress !!

**Objective**

Benchmark the performance of pre-trained models on the CIFAR-10 dataset, which includes :
1. AlexNet
2. Vgg
3. ResNet
4. SqueezeNet
5. DenseNet and
6. Inception v3

*Note*: The benchmarks might differ if you use different version of PyTorch and torchvision. Also there is a stochasticity introduced during the selection of validation set.

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

import torchvision.models as models
from torchvision.utils import make_grid
from torchvision import models
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import matplotlib.pyplot as plt
import time
import copy

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  0.4.1
Torchvision Version:  0.2.1


The torchvision library offers a lot of state of the art neural network models which come pretrained. Lets take a look at them. For the purpose of benchmarking we will limit ourselves to few of them.

In [2]:
available_model_names = sorted(name for name in models.__dict__
                     if name.islower() and not name.startswith("__")
                     and callable(models.__dict__[name]))
print([model for model in available_model_names])

['alexnet', 'densenet121', 'densenet161', 'densenet169', 'densenet201', 'inception_v3', 'resnet101', 'resnet152', 'resnet18', 'resnet34', 'resnet50', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    print('Machine supports cuda')
else:
    print('Machine does not have cuda, falling back on cpu')

Machine does not have cuda, falling back on cpu


In [4]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_names = ['resnet']#, 'alexnet', 'vgg', 'squeezenet', 'densenet', 'inception']

# Number of classes in the dataset
num_classes = 10

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for 
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model, 
#   when True we only update the reshaped layer params
feature_extract = True

In [5]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [6]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet121
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Print the model we just instantiated
#print(model_ft)

## Load the data

Let's load in the data. Pre-trained models also have to be normalized. We will be creating a training set and a validation set (by default in a 6:4 ratio). Minimal data transforms are being done, to ensure all models perform well. 

In [7]:
def load_data_pre_process(input_size, val_split=0.4):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225])
    data_transforms = {
        'train' : transforms.Compose(
        [transforms.RandomResizedCrop(input_size),
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         normalize
        ]),
        'val' : transforms.Compose(
        [transforms.Resize(input_size),
         transforms.CenterCrop(input_size),
         transforms.ToTensor(),
         normalize
        ])
    }
    
    img_datasets = {x: datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transforms[x]) for x in ['train', 'val'] }
    
    ## define our indices --  we want a 6:4 split
    num_train = len(img_datasets['train'])
    indices = list(range(num_train))
    
    # Random, non-contiguous split
    validation_idx = np.random.choice(indices, size=int(num_train*val_split), replace=False)
    train_idx = list(set(indices) - set(validation_idx))
    
    sampler_dict = {'train' : SubsetRandomSampler(train_idx),
                   'val' : SubsetRandomSampler(validation_idx)}
    dataloaders_dict = {x: torch.utils.data.DataLoader(img_datasets[x], batch_size=batch_size,
                                                       num_workers=4, sampler=sampler_dict[x]) 
                        for x in ['train', 'val']
                       }
    return dataloaders_dict

## Create the optimizer

In [8]:
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are 
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.

def create_optimizer(model_ft):
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    # Observe that all parameters are being optimized
    optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
    return optimizer_ft

# Training the model

In [9]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    perf_hist = {'train_acc' : [], 'train_loss' : [], 'val_acc': [], 'val_loss': []}

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            if phase == 'train':
                perf_hist['train_acc'] = perf_hist['train_acc'].append(epoch_acc)
                perf_hist['train_loss'] = perf_hist['train_loss'].append(epoch_loss)
            else:
                perf_hist['val_acc'] = perf_hist['val_acc'].append(epoch_acc)
                perf_hist['val_loss'] = perf_hist['val_loss'].append(epoch_loss)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    perf_hist['time'] = time_elapsed
    return model, perf_hist

In [10]:
def benchmark_models(model_names):
    """ Benchmark the different models
    """
    model_hist = []
    for model in model_names:
        # Initialize the model for this run
        model_ft, input_size = initialize_model(model, num_classes, feature_extract, use_pretrained=True)
        # Send the model to GPU
        model_ft = model_ft.to(device)
        params_to_update = model_ft.parameters()
        
        optimizer_ft = create_optimizer(model_ft)
        # Setup the loss fxn
        criterion = nn.CrossEntropyLoss()
        dataloaders_dict = load_data_pre_process(input_size, val_split=0.4)
        # Train and evaluate
        print(f'Starting training of {model} model')
        model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, 
                                     is_inception=(model=="inception"))
        hist['model'] = model
        model_hist.append(hist)

In [ ]:
benchmark_models(model_names)

Params to learn:
	 fc.weight
	 fc.bias
Files already downloaded and verified
Files already downloaded and verified
Starting training of resnet model
Epoch 0/14
----------
